# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# Loading and reviewing Dataset

In [3]:
apps_df = pd.read_csv('Play Store Data.csv')
reviews_df = pd.read_csv('User Reviews.csv')
print(apps_df.head())
print(reviews_df.head())

                                                 App        Category  Rating  \
0     Photo Editor & Candy Camera & Grid & ScrapBook  ART_AND_DESIGN     4.1   
1                                Coloring book moana  ART_AND_DESIGN     3.9   
2  U Launcher Lite – FREE Live Cool Themes, Hide ...  ART_AND_DESIGN     4.7   
3                              Sketch - Draw & Paint  ART_AND_DESIGN     4.5   
4              Pixel Draw - Number Art Coloring Book  ART_AND_DESIGN     4.3   

  Reviews  Size     Installs  Type Price Content Rating  \
0     159   19M      10,000+  Free     0       Everyone   
1     967   14M     500,000+  Free     0       Everyone   
2   87510  8.7M   5,000,000+  Free     0       Everyone   
3  215644   25M  50,000,000+  Free     0           Teen   
4     967  2.8M     100,000+  Free     0       Everyone   

                      Genres      Last Updated         Current Ver  \
0               Art & Design   January 7, 2018               1.0.0   
1  Art & Design;Pretend 

# Data Cleaning 

In [4]:
# Handling missing values and duplicates
apps_df= apps_df.dropna(subset=['Rating'])
for column in apps_df.columns:
    apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)
apps_df.drop_duplicates(inplace=True)
apps_df=apps_df[apps_df['Rating'] <= 5]
reviews_df.dropna(subset=['Translated_Review'], inplace=True)

# Convert the 'installs' column to numeric by removing the '+' and ',' characters
apps_df['Installs']=apps_df['Installs'].str.replace('+', '').str.replace(',', '').astype(int)

# Convert the 'Price' column to numeric by removing the '$' character
apps_df['Price'] = apps_df['Price'].str.replace('$', '').astype(float)
merged_df = pd.merge(apps_df, reviews_df, on='App', how='inner')
merged_df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_6220\1425243837.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[column].fillna(apps_df[column].mode()[0],inplace=True)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800,0.900000


# Data Transmission

In [5]:
# Convert the 'Size' column to numeric by handling 'M' and 'k' suffixes
def convert_size (size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'k' in size:
        return float(size.replace('k', '')) /1024
    else:
        return np.nan
apps_df['Size'] = apps_df['Size'].apply(convert_size)

# Convert the 'Reviews' column to integer
apps_df['Reviews']= apps_df['Reviews'].astype(int)

# Apply log transformation to 'Installs' and 'Reviews' columns
apps_df['Log_Installs'] = np.log1p(apps_df['Installs'])
apps_df['Log_Riviews'] = np.log1p(apps_df['Reviews'])

# Create a new column 'Rating_Group' based on the 'Rating' column
def rating_group(rating):
    if rating >= 4:
        return 'Top rated app'
    elif rating >= 3:
        return 'Above average'
    elif rating >= 2:
        return 'Average'
    else:
        return 'Below average'
apps_df['Rating_Group'] = apps_df['Rating'].apply(rating_group)

# Create a new column 'Revenue' by multiplying 'Installs' and 'Price'
apps_df['Revenue'] = apps_df['Installs'] * apps_df['Price']

# Sentiment Analysis on user reviews
sia = SentimentIntensityAnalyzer()
# Calculate sentiment scores for each review
reviews_df['Sentiment_Score'] = reviews_df['Translated_Review'].apply(lambda x: sia.polarity_scores(x)['compound'])

# Convert 'Last Updated' column to datetime format
apps_df['Last Updated']=pd.to_datetime(apps_df['Last Updated'], errors='coerce')

# Extract year from 'Last Updated' column
apps_df['Year']=apps_df['Last Updated'].dt.year

# Plotly Graphs

In [6]:
# Create directory for HTML files if it doesn't exist
html_files_path="./"
if not os.path.exists(html_files_path):
    os.makedirs(html_files_path)
    
# Initialize a variable to hold all plot containers
plot_containers=""

# save each plotly figure to an html file
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    file_path=os.path.join(html_files_path, filename)
    html_content = pio.to_html(fig, full_html=False, include_plotlyjs='inline')
    # Append the plot and its insight to the plot_containers variable 
    plot_containers += f"""
    <div class="plot-container" id="{filename}" onclick="openPlot('{filename}')">
    <div class="plot">{html_content}</div>
    <div class="insight">{insight}</div>
    </div>
    """
    fig.write_html(file_path, full_html=False, include_plotlyjs='inline')
    
# Common plot settings
plot_width=400
plot_height=300
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

# Figure 1
category_counts=apps_df['Category'].value_counts().nlargest(10)
fig1=px.bar(
    x=category_counts.index,
    y=category_counts.values,
    labels={'x':'Category', 'y':'Count'},
    title='Top Categories on Play Store',
    color=category_counts.index,
    color_discrete_sequence=px.colors.sequential.Plasma,
    width=400,
    height=300
)
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig1,"Category Graph 1.html","The top categories on the Play Store are dominated by tools, entertainment, and productivity apps")

# Figure 2
type_counts=apps_df['Type'].value_counts()
fig2=px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title='App types Distribution',
    color_discrete_sequence=px.colors.sequential.RdBu,
    width=400,
    height=300
)
fig2.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig2,"Type Graph 2.html","Most apps on the Playstore are free, indicating a strategy to attract users first and monetize through ads or inapp purchases.")

# Figure 3
fig3=px.histogram(
    apps_df,
    x='Rating',
    nbins=20,
    title='Rating Distribution',
    color_discrete_sequence=["#636EFA"],
    width=400,
    height=300
)
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig3,"Rating Graph 3.html","Ratings are skewed towards higher values, sugessting that most apps are favorable by users ")

# Figure 4
sentiment_counts=reviews_df['Sentiment_Score'].value_counts()
fig4=px.bar(
    x=sentiment_counts.index,
    y=sentiment_counts.values,
    labels={'x':'Sentiment Score', 'y':'Count'},
    title='Sentiment Distribution',
    color=sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu,
    width=400,
    height=300
)
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig4,"Sentiment Graph 4.html","Sentiments in reviews show a mix of positive and negative feedback, with a slight lean towards positive sentiments")

# Figure 5
install_by_category=apps_df.groupby('Category')['Installs'].sum().nlargest(10)
fig5=px.bar(
    x=install_by_category.values,
    y=install_by_category.index,
    orientation='h',
    labels={'x':'Installs', 'y':'Category'},
    title='Installs by Category',
    color=install_by_category.index,
    color_discrete_sequence=px.colors.sequential.Blues,
    width=400,
    height=300
)
fig5.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig5,"Installs Graph 5.html","The categoris with the most installs are social and communication apps, reflecting their broad appeal and daily usage")

# Figure 6
updates_per_year=apps_df['Last Updated'].dt.year.value_counts().sort_index()
fig6=px.line(
    x=updates_per_year.index,
    y=updates_per_year.values,
    labels={'x':'Year', 'y':'Number of Updates'},
    title='Number of Updates Over the Years',
    color_discrete_sequence=['#AB63FE'],
    width=plot_width,
    height=plot_height
)
fig6.update_layout(
    plot_bgcolor=plot_bg_color,
    paper_bgcolor=plot_bg_color,
    font_color=text_color,
    title_font=title_font,
    xaxis=dict(title_font=axis_font),
    yaxis=dict(title_font=axis_font),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig6,"Updates Graph 6.html", "Updades have been increasing over the years, indicating that developers are actively maintaining and improving their apps.")

# Figure 7
revenue_by_category=apps_df.groupby('Category')['Revenue'].sum().nlargest(10)
fig7=px.bar(
    x=revenue_by_category.index,
    y=revenue_by_category.values,
    labels={'x':'Category', 'y':'Revenue'},
    title='Revenue by Category',
    color=revenue_by_category.index,
    color_discrete_sequence=px.colors.sequential.Greens,
    width=400,
    height=300
)
fig7.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig7,"Revenue Graph 7.html","Categories such as Family and Lifestyle lead in revenue generation, indicating their monetization potential.")

# Figure 8
genre_counts=apps_df['Genres'].str.split(';',expand=True).stack().value_counts().nlargest(10)
fig8=px.bar(
    x=genre_counts.index,
    y=genre_counts.values,
    labels={'x':'Genre', 'y':'Count'},
    title='Top Genres',
    color=genre_counts.index,
    color_discrete_sequence=px.colors.sequential.OrRd,
    width=400,
    height=300
)
fig8.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig8,"Genre Graph 8.html","Action and Entertainment genres are the most common, reflecting users' prefrence for engaging and easy-to-play games.")

# Figure 9
fig9=px.scatter(
    apps_df,
    x='Last Updated',
    y='Rating',
    color='Type',
    title='Impact of Last Update on Rating',
    color_discrete_sequence=px.colors.qualitative.Vivid,
    width=400,
    height=300
)
fig9.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig9,"Update X Rating Graph 9.html","The Scatter plot shows a weak correlation between the last update and ratings, suggesting that more frequent updates don't always result in better ratings.")

# Figure 10
fig10=px.box(
    apps_df,
    x='Type',
    y='Rating',
    color='Type',
    title='Rating for Paid vs Free Apps',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    width=400,
    height=300
)
fig10.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    title_font={'size':16},
    xaxis=dict(title_font={'size':12}),
    yaxis=dict(title_font={'size':12}),
    margin=dict(l=10, r=10, t=30, b=10)
)
save_plot_as_html(fig10,"Paid Free Graph 10.html","Paid apps generally have higher ratings compared to free apps,suggesting that users expect higher quality from apps they pay for.")

# Split the plot containers into individual plots for display
plot_containers_split=plot_containers.split('</div>')

# Wrap each plot container in a div for better styling and interaction
if len(plot_containers_split) > 1:
    final_plot=plot_containers_split[-2] + '</div>'
else:
    final_plot=plot_containers

# Dashboard Creation

In [7]:
# Create the final HTML dashboard
dashboard_html="""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Google Play Store Review Analysis</title>
<style>
    body {{
        font-family: Arial, sans-serif;
        background-color: #333;
        color: #fff;
        margin: 0;
        padding: 0;
        }}
    .header {{
        display: flex;
        align-items: center;
        justify-content: center;
        padding: 20px;
        background-color: #444;
        }}
    .header img {{
        margin: 0 10px;
        height: 50px;
        }}
    .container {{
        display: flex;
        flex-wrap: wrap;
        justify-content: center;
        padding: 20px;
        }}
    .plot-container {{
        border: 2px solid #555;
        margin: 10px;
        padding: 10px;
        width: {plot_width}px;
        height: {plot_height}px;
        overflow: hidden;
        position: relative;
        cursor: pointer;
        }}
    .insight {{
        display: none;
        position: absolute;
        right: 10px;
        top: 10px;
        background-color: rgba(0, 0, 0, 0.7);
        padding: 5px;
        border-radius: 5px;
        color: #fff;
        }}
    .plot-container:hover .insight {{
        display: block;
        }}
</style>
<script>
    function openPlot(filename) {{
        window.open(filename, '_blank');
        }}
</script>
</head>
<body>
    <div class="header">
      <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Logo_2013_Google.png/800px-Logo_2013_Google.png" alt="Google Logo" />
      <h1>Google Play Store Review Analysis</h1>
      <img src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Google_Play_2022_logo.svg" alt="Google Play Store Logo" />
    </div>
    <div class="container">
        {plots}        
    </div>
</body>
</html>
"""

# Combine all plot containers into the final HTML
final_html=dashboard_html.format(plots=plot_containers, plot_width=plot_width, plot_height=plot_height)

# Save the final HTML to a file
dashboard_path=os.path.join(html_files_path,"web page.html")

# Write the final HTML to a file
with open(dashboard_path, 'w', encoding='utf-8') as f:
    f.write(final_html)
  
# Open the dashboard in the default web browser  
webbrowser.open('file://' + os.path.realpath(dashboard_path))

True